In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
'''

from torch.utils.data import DataLoader

In [2]:
!wget -P ../data/raw https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Beijing%20PM2.5/PRSA_data_2010.1.1-2014.12.31.csv

--2022-03-10 03:13:53--  https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Beijing%20PM2.5/PRSA_data_2010.1.1-2014.12.31.csv
Resolving code.datasciencedojo.com (code.datasciencedojo.com)... 167.99.111.153
Connecting to code.datasciencedojo.com (code.datasciencedojo.com)|167.99.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1966669 (1.9M) [text/plain]
Saving to: ‘../data/raw/PRSA_data_2010.1.1-2014.12.31.csv.2’

PRSA_data_2010.1.1- 100%[===================>]   1.88M  1.61MB/s    in 1.2s    

2022-03-10 03:13:55 (1.61 MB/s) - ‘../data/raw/PRSA_data_2010.1.1-2014.12.31.csv.2’ saved [1966669/1966669]



In [3]:
!mkdir ../data/processed/beijing_pollution

mkdir: cannot create directory ‘../data/processed/beijing_pollution’: File exists


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
df = pd.read_csv('../data/raw/PRSA_data_2010.1.1-2014.12.31.csv')
df_cleaned = df.copy()
df_cleaned.drop('No', axis=1, inplace=True)
df_cleaned.dropna(inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

In [6]:
num_cols = ['year', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
cat_cols = ['month', 'day', 'hour', 'cbwd']

sc = StandardScaler()
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

ohe = OneHotEncoder(sparse=False)
X_cat = pd.DataFrame(ohe.fit_transform(df_cleaned[cat_cols]))
X_cat.columns = ohe.get_feature_names(cat_cols)

df_cleaned.drop(cat_cols, axis=1, inplace=True)
X = pd.concat([df_cleaned, X_cat], axis=1)

In [7]:
from src.data.sets import split_sets_by_time, save_sets

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_by_time(X, target_col='pm2.5', test_ratio=0.2)

save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='../data/processed/beijing_pollution/')

In [8]:
from src.models.null import NullModel
from src.models.performance import print_reg_perf

baseline_model = NullModel()
y_base = baseline_model.fit_predict(y_train)
print_reg_perf(y_base, y_train, set_name='Training')

RMSE Training: 92.82545840756482
MAE Training: 69.67082209440568


In [19]:
from src.models.pytorch import PytorchRegression
from src.models.pytorch import get_device

model = PytorchRegression(X_train.shape[1])
device = get_device()
model.to(device)

PytorchRegression(
  (layer_1): Linear(in_features=78, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)

In [28]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [33]:
from src.models.pytorch import train_regression, test_regression

N_EPOCHS = 5
BATCH_SIZE = 32
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(N_EPOCHS):
    train_loss, train_rmse = train_regression(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_rmse = test_regression(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\tLoss: {train_loss:.4f}\t|\tRMSE: {train_rmse:.1f}')
    print(f'\t(valid)\tLoss: {valid_loss:.4f}\t|\tRMSE: {valid_rmse:.1f}')

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0
	(train)	Loss: 350.2396	|	RMSE: 18.7
	(valid)	Loss: 246.5566	|	RMSE: 15.7
Epoch: 1
	(train)	Loss: 273.5814	|	RMSE: 16.5
	(valid)	Loss: 237.1896	|	RMSE: 15.4
Epoch: 2
	(train)	Loss: 272.4216	|	RMSE: 16.5
	(valid)	Loss: 237.3426	|	RMSE: 15.4
Epoch: 3
	(train)	Loss: 272.3799	|	RMSE: 16.5
	(valid)	Loss: 236.0889	|	RMSE: 15.4
Epoch: 4
	(train)	Loss: 272.2827	|	RMSE: 16.5
	(valid)	Loss: 237.7741	|	RMSE: 15.4


In [34]:
torch.save(model, "../models/pytorch_reg_polution.pt")

,year,DEWP,TEMP,PRES,Iws,Is,Ir,month_1,month_2,month_3,...,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
0,-1.443355,-1.229791,-1.347143,0.345329,-0.444944,-0.071057,-0.137408,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1.443355,-1.160508,-1.347143,0.345329,-0.427007,-0.071057,-0.137408,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.443355,-0.883375,-1.429278,0.442411,-0.409069,-0.071057,-0.137408,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.443355,-0.606241,-1.429278,0.539493,-0.372993,1.212862,-0.137408,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1.443355,-0.606241,-1.429278,0.539493,-0.355055,2.496781,-0.137408,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
